In [1]:
'''更新日志10July
单通道测试。
打脸了，不用原始牙的数据也能完成评分。
'''

'更新日志10July\n单通道测试。\n'

In [2]:
'''
双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。
输入：预备体的体素模型
网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出
'''

'\n双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。\n输入：预备体的体素模型\n网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出\n'

In [3]:
'''
# 安装环境
pip install trimesh
'''

'\n# 安装环境\npip install trimesh\n'

In [4]:
'''
# 安装环境
pip install torch torchvision torchaudio
'''

'\n# 安装环境\npip install torch torchvision torchaudio\n'

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import trimesh

In [6]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  



In [7]:
# 定义数据集
class TeethDataset(Dataset):
    def __init__(self, score_file_path, voxel_size=32):
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data = mesh_to_voxel(mesh, self.voxel_size)
        return torch.tensor(voxel_data[np.newaxis, :], dtype=torch.float32), torch.tensor([score], dtype=torch.float32)

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(score_file_path, voxel_size=32)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [8]:
# 定义模型
class Teeth3DCNN(nn.Module):
    def __init__(self):
        super(Teeth3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1)  # 单通道输入
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)  # 体素尺寸为32*32*32，经过两次pooling变为8*8*8
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建模型
model = Teeth3DCNN()
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

In [9]:
# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(dataloader)}")


Epoch 1/30, Loss: 851.8548606571399
Epoch 2/30, Loss: 272.336025739971
Epoch 3/30, Loss: 274.40824696892184
Epoch 4/30, Loss: 263.5247444855539
Epoch 5/30, Loss: 272.32252105913665
Epoch 6/30, Loss: 310.97679108067564
Epoch 7/30, Loss: 323.8178389197902
Epoch 8/30, Loss: 285.33358232598556
Epoch 9/30, Loss: 244.26785830447548
Epoch 10/30, Loss: 241.80397375006424
Epoch 11/30, Loss: 222.27285533202323
Epoch 12/30, Loss: 234.78255083686426
Epoch 13/30, Loss: 219.62228905527215
Epoch 14/30, Loss: 278.8621797059712
Epoch 15/30, Loss: 197.170098285926
Epoch 16/30, Loss: 202.88164098639237
Epoch 17/30, Loss: 172.44623043662622
Epoch 18/30, Loss: 170.7503856859709
Epoch 19/30, Loss: 162.89898676621286
Epoch 20/30, Loss: 158.78864709954513
Epoch 21/30, Loss: 137.5020196312352
Epoch 22/30, Loss: 121.86741337023284
Epoch 23/30, Loss: 106.65802968175788
Epoch 24/30, Loss: 81.53537810476203
Epoch 25/30, Loss: 82.55251262062474
Epoch 26/30, Loss: 74.38074779510498
Epoch 27/30, Loss: 61.363033129980

In [10]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}')


Tooth 1: Score = 72.3387451171875
Tooth 2: Score = 65.14057922363281
Tooth 3: Score = 66.97459411621094
Tooth 4: Score = 67.66551971435547
Tooth 5: Score = 70.59071350097656
Tooth 6: Score = 62.392051696777344
Tooth 7: Score = 69.6642074584961
Tooth 8: Score = 72.06585693359375
Tooth 9: Score = 75.99604797363281
Tooth 10: Score = 70.83352661132812
Tooth 11: Score = 50.20680618286133
Tooth 12: Score = 58.48654556274414
Tooth 13: Score = 58.95806884765625
Tooth 14: Score = 63.12580108642578
Tooth 15: Score = 51.276241302490234
Tooth 16: Score = 39.08396911621094
Tooth 17: Score = 53.69865417480469
Tooth 18: Score = 58.24885559082031
Tooth 19: Score = 65.99652099609375
Tooth 20: Score = 56.61967849731445
Tooth 21: Score = 64.19806671142578
Tooth 22: Score = 63.921287536621094
Tooth 23: Score = 66.824462890625
Tooth 24: Score = 67.26507568359375
Tooth 25: Score = 59.27981185913086
Tooth 26: Score = 59.40177917480469
Tooth 27: Score = 64.4385986328125
Tooth 28: Score = 63.4124755859375
Toot

In [11]:
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )

Tooth 1: Score = 72.3387451171875 Err = -2.6612548828125
Tooth 2: Score = 65.14057922363281 Err = -2.8594207763671875
Tooth 3: Score = 66.97459411621094 Err = -3.0254058837890625
Tooth 4: Score = 67.66551971435547 Err = -1.3344802856445312
Tooth 5: Score = 70.59071350097656 Err = -5.4092864990234375
Tooth 6: Score = 62.392051696777344 Err = -1.6079483032226562
Tooth 7: Score = 69.6642074584961 Err = 0.6642074584960938
Tooth 8: Score = 72.06585693359375 Err = -0.93414306640625
Tooth 9: Score = 75.99604797363281 Err = -1.0039520263671875
Tooth 10: Score = 70.83352661132812 Err = -6.166473388671875
Tooth 11: Score = 50.20680618286133 Err = -3.793193817138672
Tooth 12: Score = 58.48654556274414 Err = -3.5134544372558594
Tooth 13: Score = 58.95806884765625 Err = -0.04193115234375
Tooth 14: Score = 63.12580108642578 Err = 0.12580108642578125
Tooth 15: Score = 51.276241302490234 Err = -5.723758697509766
Tooth 16: Score = 39.08396911621094 Err = -8.916030883789062
Tooth 17: Score = 53.69865417